# **2. Black-Scholes 模型下的欧式期权**
- [2.1 Black-Scholes 模型的基本假设](#2.1-Black-Scholes-模型的基本假设)
- [2.2 风险中性测度与资产定价](#2.2-风险中性测度与资产定价)
- [2.3 无套利对冲与 Black-Scholes PDE](#2.3-无套利对冲与-Black-Scholes-PDE)
- [2.4 Black-Scholes 公式的解析解](#2.4-Black-Scholes-公式的解析解)
- [2.5 Python 实现](#2.5-Python-实现)
- [2.6 附录](#2.6-附录)

---


### **2.1 Black-Scholes 模型的基本假设**
Black-Scholes 模型基于以下市场假设：
- **市场无摩擦**：不存在交易成本、税收或卖空限制，资产可无限分割。
- **无风险利率恒定**：投资者可按固定利率 $r$ 进行借贷。
- **资产价格遵循几何布朗运动**：
  $$
  dS_t = \mu S_t dt + \sigma S_t dW_t
  $$
  其中，$\mu$ 为期望收益率，$\sigma$ 为波动率，$W_t$ 为标准布朗运动。
- **市场无套利**：不存在无风险套利机会。
- **欧式期权**：仅能在到期日 $T$ 行权。
- **资产不支付股息**：期权存续期内无现金流分配。

---

### **2.2 风险中性测度与资产定价**
根据 Girsanov 定理，可以找到一个新的概率测度——**风险中性测度** $Q$，使得资产价格的动态变为：
$$
dS_t = r S_t dt + \sigma S_t dW_t^Q
$$
其中 $W_t^Q$ 是在 $Q$ 测度下的标准布朗运动。

在风险中性世界中，资产的期望收益率等于无风险利率 $r$，所有衍生品的价格可表示为未来现金流的期望折现：
$$
e^{-r(T-t)} \mathbb{E}_Q[S_T] = S_t
$$
这一理论意味着期权定价可以通过计算其未来现金流的折现值来求得。

---

### **2.3 无套利对冲与 Black-Scholes PDE**

设期权价格 $C(S_t, t)$ 是资产价格 $S_t$ 和时间 $t$ 的函数，利用 **伊藤引理（Itô's Lemma）**，其变化服从：
$$
dC = \left( \frac{\partial C}{\partial t} + \mu S_t \frac{\partial C}{\partial S} + \frac{1}{2} \sigma^2 S_t^2 \frac{\partial^2 C}{\partial S^2} \right) dt + \sigma S_t \frac{\partial C}{\partial S} dW_t
$$

构造无风险投资组合：
$$
\Pi = C - \Delta S_t
$$

选择：$\Delta = \frac{\partial C}{\partial S}$（证明见附录）以消除随机项 $dW_t$，则投资组合的变化为：
$$
d\Pi = \left( \frac{\partial C}{\partial t} + \frac{1}{2} \sigma^2 S_t^2 \frac{\partial^2 C}{\partial S^2} \right) dt
$$

在 **无套利原则** 下，该无风险投资组合的收益率必须等于无风险利率 $r$，即：
$$
\frac{d\Pi}{\Pi} = r dt
$$
或者等价地：
$$
d\Pi = r\Pi dt
$$

代入 $\Pi = C - \frac{\partial C}{\partial S} S_t$，得：
$$
\left( \frac{\partial C}{\partial t} + \frac{1}{2} \sigma^2 S_t^2 \frac{\partial^2 C}{\partial S^2} \right) dt = r \left( C - \frac{\partial C}{\partial S} S \right) dt
$$

整理后得到 **Black-Scholes 偏微分方程（PDE）**：
$$
\frac{\partial C}{\partial t} + \frac{1}{2} \sigma^2 S^2 \frac{\partial^2 C}{\partial S^2} + r S \frac{\partial C}{\partial S} - rC = 0
$$

该方程描述了欧式期权的价格如何随时间和资产价格的变化而演化。

---

### **2.4 Black-Scholes 公式的解析解**

对于欧式 **看涨期权（Call Option）**，假设边界条件：
$$
C(S_T, T) = \max(S_T - K, 0)
$$
求解 Black-Scholes 偏微分方程（PDE）可得其解析解。值得注意的是，该解析解通常通过 **热力方程变换** 方法求解，但 **Black-Scholes 公式的解法并非唯一**。若读者对不同求解方法感兴趣，可以参考 [Frouah 的 Black-Scholes 公式推导](https://frouah.com/finance%20notes/Black%20Scholes%20Formula.pdf)，这篇文章中提供了多种BS公式的求解方法。

Black-Scholes 看涨期权定价公式：
$$
C(S_t, t) = S_t N(d_1) - K e^{-r(T-t)} N(d_2)
$$
其中：
$$
d_1 = \frac{\ln(S_t / K) + (r + \sigma^2 / 2)(T-t)}{\sigma \sqrt{T-t}}, \quad
d_2 = d_1 - \sigma \sqrt{T-t}
$$
$N(d)$ 为 **标准正态分布的累积分布函数**。

对于欧式 **看跌期权（Put Option）**，可利用 **看涨-看跌平价（Put-Call Parity）**：
$$
P(S_t, t) = C(S_t, t) + K e^{-r(T-t)} - S_t
$$
或者直接求解 PDE，得出 Black-Scholes 看跌期权定价公式：

$$
P(S_t, t) = K e^{-r(T-t)} N(-d_2) - S_t N(-d_1)
$$

---

### **2.5 Python 实现**

下面我们分别计算看涨/看跌欧式期权，定义 black_scholes 函数

In [2]:
import numpy as np
from scipy.stats import norm

def black_scholes(S, K, T, r, sigma, option_type="call"):
    """
    Black-Scholes 期权定价公式计算欧式看涨/看跌期权价格
    
    参数：
    S : float  -  当前资产价格 (Spot Price)
    K : float  -  期权执行价格 (Strike Price)
    T : float  -  到期时间 (Time to Maturity, in years)
    r : float  -  无风险利率 (Risk-Free Interest Rate)
    sigma : float  -  资产价格波动率 (Volatility)
    option_type : str  -  "call" 表示看涨期权, "put" 表示看跌期权
    
    返回：
    期权理论价格 (float)
    """
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if option_type == "call":
        return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif option_type == "put":
        return K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    else:
        raise ValueError("option_type 必须是 'call' 或 'put'")


假设当前资产价格 $S = 100$，执行价格 $K = 100$，到期时间 $T = 1$ 年，无风险利率 $r = 5\%$，波动率 $\sigma = 20\%$。该场景模拟了一个 **1 年到期的欧式期权**，标的资产当前价格与执行价格相同，市场无风险利率为 5%，资产的年化波动率为 20%。我们使用 **Black-Scholes 公式** 计算该期权的理论价格。

In [3]:
# 参数
S = 100      # 现价
K = 100      # 执行价格
T = 1        # 剩余时间（年）
r = 0.05     # 无风险利率
sigma = 0.2  # 波动率

# 计算欧式期权价格
call_price = black_scholes(S, K, T, r, sigma, option_type="call")
put_price = black_scholes(S, K, T, r, sigma, option_type="put")

print(f"欧式看涨期权价格 (Call Option Price): {call_price:.4f}")
print(f"欧式看跌期权价格 (Put Option Price): {put_price:.4f}")


欧式看涨期权价格 (Call Option Price): 10.4506
欧式看跌期权价格 (Put Option Price): 5.5735


---

### **2.6 附录：$\Delta = \frac{\partial C}{\partial S}$ 证明**

### **Itô 引理与期权价格变化**
根据 **伊藤引理（Itô's Lemma）**，期权价格 $C(S_t, t)$ 的变化服从：
$$
dC = \left( \frac{\partial C}{\partial t} + \mu S_t \frac{\partial C}{\partial S} + \frac{1}{2} \sigma^2 S_t^2 \frac{\partial^2 C}{\partial S^2} \right) dt + \sigma S_t \frac{\partial C}{\partial S} dW_t
$$
其中：
- $\frac{\partial C}{\partial t}$ 是期权价格对时间的变化率，
- $\frac{\partial C}{\partial S}$ 是期权价格对资产价格的敏感度（即 **Delta**），
- $\frac{\partial^2 C}{\partial S^2}$ 是 **Gamma**，反映二阶变化，
- $\mu S_t \frac{\partial C}{\partial S}$ 是 **漂移项**，
- $\sigma S_t \frac{\partial C}{\partial S} dW_t$ 是 **随机波动项**。

### **构造无风险投资组合**
构造投资组合 $\Pi$，包含：
- **持有 1 份期权 $C$**，
- **做空 $\Delta$ 份标的资产 $S_t$**，即：
$$
\Pi = C - \Delta S_t
$$
该组合的微小变动为：
$$
d\Pi = dC - \Delta dS_t
$$

假设标的资产 $S_t$ 服从 **几何布朗运动**：
$$
dS_t = \mu S_t dt + \sigma S_t dW_t
$$
将 $dS_t$ 代入 $d\Pi$：
$$
d\Pi = \left( \frac{\partial C}{\partial t} + \mu S_t \frac{\partial C}{\partial S} + \frac{1}{2} \sigma^2 S_t^2 \frac{\partial^2 C}{\partial S^2} \right) dt + \sigma S_t \frac{\partial C}{\partial S} dW_t - \Delta (\mu S_t dt + \sigma S_t dW_t)
$$
整理后：
$$
d\Pi = \left( \frac{\partial C}{\partial t} + \mu S_t \frac{\partial C}{\partial S} + \frac{1}{2} \sigma^2 S_t^2 \frac{\partial^2 C}{\partial S^2} - \Delta \mu S_t \right) dt + (\sigma S_t \frac{\partial C}{\partial S} - \Delta \sigma S_t) dW_t
$$

**消除随机项以实现无风险**
为了使投资组合 **无风险**，我们必须消除 $dW_t$ 相关项，使其系数为 0，即：
$$
\Delta = \frac{\partial C}{\partial S}
$$
这样，随机项消失：
$$
\sigma S_t \left( \frac{\partial C}{\partial S} - \Delta \right) dW_t = 0
$$
最终，投资组合的变化变为：
$$
d\Pi = \left( \frac{\partial C}{\partial t} + \frac{1}{2} \sigma^2 S_t^2 \frac{\partial^2 C}{\partial S^2} \right) dt
$$
这是一个**确定性的变化**，意味着该组合是 **无风险的**。
**Delta（$\Delta$）** 是通过设定组合中的标的资产持仓量，使其 **完全对冲掉期权价格变动中的随机项** 来获得的。最终得到：
$$
\Delta = \frac{\partial C}{\partial S}
$$
这就是期权定价理论中的 **Delta 对冲（Delta Hedging）方法**。
